# Build angular format dataset

## Total_data

In [5]:
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm
import re

def check_angular_convention(msg, has_space=True, strict=True):
    types = '((build)|(ci)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test)|(chore))'
    # types = '((perf))'
    if has_space:
        pattern = f'{types}\\s(\\((\\s|\\S)+\\)\\s)?:\\s(\\s|\\S)+'
    else:
        pattern = f'{types}(\\((\\s|\\S)+\\))?:\\s\\S+(\\s|\\S)+'
    # if not strict:
    #     pattern = '^((build)|(ci)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test)|(chore))'
    return re.match(pattern, msg) is not None

# Define the types for the angular convention
angular_types = ['build', 'ci', 'docs', 'feat', 'fix', 'perf', 'refactor', 'style', 'test', 'chore']
# angular_types = ['perf']

# Create empty dictionaries for each language and type
languages = ['JavaScript']
dfs = {lang: {atype: [] for atype in angular_types} for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=100000, desc=lang) for lang in languages}  # Total: 10 types * 10000 each

folder_path = '../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

# files = glob.glob(os.path.join(folder_path, 'train*')) + glob.glob(os.path.join(folder_path, 'validation*'))
files = glob.glob(os.path.join(folder_path, 'train*')) + glob.glob(os.path.join(folder_path, 'validation*')) + glob.glob(os.path.join(folder_path, 'test*')) 

for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages[:]:
        lang_suffix = '.js'

        # Filter rows where language column matches the current language and additional conditions
        lang_df = df[(df['language'] == lang) & 
                     (df['mods'].apply(len) == 1) & 
                     # (df['message'].apply(lambda x: len(x.split())) <= 30) & 
                     (df['mods'].apply(lambda x: x[0]['change_type']) == 'MODIFY') &
                     (df['mods'].apply(lambda x: len(f"diff --git a/{x[0]['old_path']} b/{x[0]['new_path']} {x[0]['diff']}") <= 5000))] 
                    #  (df['mods'].apply(lambda x: ( (x[0]['old_path'].count(lang_suffix) if x[0]['old_path'] else 0) == 1 and (x[0]['new_path'].count(lang_suffix) if x[0]['new_path'] else 0) == 1 and (x[0]['old_path'].count('.json') if x[0]['old_path'] else 0) == 0 and (x[0]['new_path'].count('.json') if x[0]['new_path'] else 0) == 0 )))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            msg = row['message']
            if check_angular_convention(msg, has_space=True) or check_angular_convention(msg, has_space=False):
                diff = row['mods'][0]
                old_path = 'a/' + diff['old_path']
                new_path = 'b/' + diff['new_path']
                diff_content = diff['diff']  # assume diff_content is an empty string
                item = {
                    'msg': row['message'],
                    'diff': f"diff --git {old_path} {new_path} {diff_content}",
                    'date': row['date'],
                    'repo': row['repo']
                }

                # Find the type in the message
                for atype in angular_types:
                    if msg.startswith(atype):
                        if len(dfs[lang][atype]) < 10000:
                            dfs[lang][atype].append(item)
                            bars[lang].update(1)
                        break

            # Check if all types have reached 1000 rows
            if all(len(dfs[lang][atype]) >= 10000 for atype in angular_types):
                print(f"Reached 10000 rows for all types in {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break

    # Break out of the loop if all languages have reached the required number of rows
    if not languages:
        break

JavaScript:   0%|          | 0/100000 [00:00<?, ?it/s]

In [6]:
db_data = []

for lang, types in dfs.items():
    for atype, items in types.items():
        if items:  # Ensure there are items to split
            for item in items:
                item['type'] = atype
            db_data.extend(items)

import json
output_dir = '../data/angular_filtered/subsets'
with open(os.path.join(output_dir, f'db_data.json'), 'w') as db_file:
    json.dump(db_data, db_file, indent=4)

In [7]:
import pandas as pd
import json
# Split the data and save to files
output_dir = '../data/angular_filtered/subsets'
os.makedirs(output_dir, exist_ok=True)

for lang, types in dfs.items():
    for atype, items in types.items():
        if items:  # Ensure there are items to split
            with open(os.path.join(output_dir, f'type_db/{lang}_{atype}_db.json'), 'w') as db_file:
                json.dump(items, db_file, indent=4)

print("Data splitting and saving completed.")

Data splitting and saving completed.


In [8]:
# Initialize a dictionary to hold counts for each type
type_counts = {lang: {atype: 0 for atype in angular_types} for lang in languages}

# Iterate over the collected data to count occurrences of each type
for lang, types in dfs.items():
    for atype, items in types.items():
        type_counts[lang][atype] = len(items)

# Print the counts
for lang, counts in type_counts.items():
    print(f"Language: {lang}")
    for atype, count in counts.items():
        print(f"  Type: {atype}, Count: {count}")

Language: JavaScript
  Type: build, Count: 1076
  Type: ci, Count: 1287
  Type: docs, Count: 9672
  Type: feat, Count: 10000
  Type: fix, Count: 10000
  Type: perf, Count: 472
  Type: refactor, Count: 6574
  Type: style, Count: 883
  Type: test, Count: 3891
  Type: chore, Count: 10000


## Test_data

In [9]:
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm
import re

def check_angular_convention(msg, has_space=True, strict=True):
    types = '((build)|(ci)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test)|(chore))'
    # types = '((perf))'
    if has_space:
        pattern = f'{types}\\s(\\((\\s|\\S)+\\)\\s)?:\\s(\\s|\\S)+'
    else:
        pattern = f'{types}(\\((\\s|\\S)+\\))?:\\s\\S+(\\s|\\S)+'
    # if not strict:
    #     pattern = '^((build)|(ci)|(docs)|(feat)|(fix)|(perf)|(refactor)|(style)|(test)|(chore))'
    return re.match(pattern, msg) is not None

# Define the types for the angular convention
angular_types = ['build', 'ci', 'docs', 'feat', 'fix', 'perf', 'refactor', 'style', 'test', 'chore']
# angular_types = ['perf']

# Create empty dictionaries for each language and type
languages = ['JavaScript']
dfs = {lang: {atype: [] for atype in angular_types} for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=2000, desc=lang) for lang in languages}  # Total: 10 types * 10000 each

folder_path = '../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

# files = glob.glob(os.path.join(folder_path, 'train*')) + glob.glob(os.path.join(folder_path, 'validation*')) + glob.glob(os.path.join(folder_path, 'test*')) 
files = glob.glob(os.path.join(folder_path, 'test*')) 

for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages[:]:
        lang_suffix = '.js'

        # Filter rows where language column matches the current language and additional conditions
        lang_df = df[(df['language'] == lang) & 
                     (df['mods'].apply(len) == 1) & 
                     # (df['message'].apply(lambda x: len(x.split())) <= 30) & 
                     (df['mods'].apply(lambda x: x[0]['change_type']) == 'MODIFY') &
                     (df['mods'].apply(lambda x: len(f"diff --git a/{x[0]['old_path']} b/{x[0]['new_path']} {x[0]['diff']}") <= 5000))] 
                    #  (df['mods'].apply(lambda x: ( (x[0]['old_path'].count(lang_suffix) if x[0]['old_path'] else 0) == 1 and (x[0]['new_path'].count(lang_suffix) if x[0]['new_path'] else 0) == 1 and (x[0]['old_path'].count('.json') if x[0]['old_path'] else 0) == 0 and (x[0]['new_path'].count('.json') if x[0]['new_path'] else 0) == 0 )))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            msg = row['message']
            if check_angular_convention(msg, has_space=True) or check_angular_convention(msg, has_space=False):
                diff = row['mods'][0]
                old_path = 'a/' + diff['old_path']
                new_path = 'b/' + diff['new_path']
                diff_content = diff['diff']  # assume diff_content is an empty string
                item = {
                    'msg': row['message'],
                    'diff': f"diff --git {old_path} {new_path} {diff_content}",
                    'date': row['date'],
                    'repo': row['repo']
                }

                # Find the type in the message
                for atype in angular_types:
                    if msg.startswith(atype):
                        if len(dfs[lang][atype]) < 200:
                            dfs[lang][atype].append(item)
                            bars[lang].update(1)
                        break

            # Check if all types have reached 1000 rows
            if all(len(dfs[lang][atype]) >= 200 for atype in angular_types):
                print(f"Reached 10000 rows for all types in {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break

    # Break out of the loop if all languages have reached the required number of rows
    if not languages:
        break

JavaScript:   0%|          | 0/2000 [00:00<?, ?it/s]

In [10]:
# Initialize a dictionary to hold counts for each type
type_counts = {lang: {atype: 0 for atype in angular_types} for lang in languages}

# Iterate over the collected data to count occurrences of each type
for lang, types in dfs.items():
    for atype, items in types.items():
        type_counts[lang][atype] = len(items)

# Print the counts
for lang, counts in type_counts.items():
    print(f"Language: {lang}")
    for atype, count in counts.items():
        print(f"  Type: {atype}, Count: {count}")

Language: JavaScript
  Type: build, Count: 200
  Type: ci, Count: 200
  Type: docs, Count: 200
  Type: feat, Count: 200
  Type: fix, Count: 200
  Type: perf, Count: 31
  Type: refactor, Count: 200
  Type: style, Count: 200
  Type: test, Count: 200
  Type: chore, Count: 200


In [11]:
test_data = []

for lang, types in dfs.items():
    for atype, items in types.items():
        if items:  # Ensure there are items to split
            for item in items:
                item['type'] = atype
            test_data.extend(items)

import json
output_dir = '../data/angular_filtered/subsets'
with open(os.path.join(output_dir, f'test_data.json'), 'w') as db_file:
    json.dump(test_data, db_file, indent=4)

# Model generation

## RACE

In [2]:
import json
from tqdm.auto import tqdm
with open('../data/angular_filtered/subsets/test_data.json') as f:
    prompt_data = json.load(f)
    
import os
os.environ['HF_HOME'] = '../models/'

from transformers import pipeline
pipe = pipeline("text2text-generation", model="JetBrains-Research/cmg-race-without-history", device=0)

In [3]:

diffs = []
generated_commit_messages = []

for commit in prompt_data:
    diff = commit['diff']
    diffs.append(diff)

for diff in tqdm(diffs, total=len(diffs), desc='Generating commit messages'):
    prompt = f"""
    The following is a diff which describes the code changes in a commit, Your task is to write a short commit message accordingly.
    {diff}
    According to the diff, the commit message should be:
    """
    generated_commit_messages.append(pipe(prompt)[0]['generated_text'])

Generating commit messages:   0%|          | 0/1831 [00:00<?, ?it/s]

C:\Users\tyfann\anaconda3\envs\llm4commit\lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1530 > 512). Running this sequence through the model will result in indexing errors
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 1100, in emit
    msg = self.format(record)
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 943, in format
    return fmt.format(record)
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__.py", line 678, in format
    record.message = record.getMessage()
  File "C:\Users\tyfann\anaconda3\envs\llm4commit\lib\logging\__init__

In [4]:

for item, msg in zip(prompt_data, generated_commit_messages):
    item['race'] = msg

output_file = '../data/angular_filtered/subsets/generation/test_race.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(prompt_data, f, ensure_ascii=False, indent=4)

# save msg to a file
with open("../data/angular_filtered/subsets/generation/test_race.txt", 'w', encoding='UTF-8') as file:
    for item in prompt_data:
        file.write(item['race'].replace('\n', '\\n').replace('\r', '\\r') + '\n')

## ChatGPT

In [5]:
from langchain import hub
import json
from tqdm import tqdm
prompt = hub.pull("tyfann/llm4commit-zeroshot")

from openai import OpenAI

client = OpenAI(
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    base_url="https://api.chatanywhere.cn/v1"
)

def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0)
    return completion.choices[0].message.content

with open('../data/angular_filtered/subsets/test_data.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    # merged_diff = '\n'.join(diff['diff'] for diff in data['diff'])
    messages = prompt.invoke(
        {"DIFF": data['diff']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    try:
        gpt_msg.append(gpt_35_api(example_prompt))
    except:
        print(index)
        gpt_msg.append("")

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_zeroshot'] = msg
import os
# output_file = '../data/chronicle/rag_baseline/zeroshot/rag_baseline_python_chatgpt.json'
output_file = '../data/angular_filtered/subsets/generation/test_gpt35_zeroshot.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

output_file = '../data/angular_filtered/subsets/generation/test_gpt35_zeroshot.txt'
with open(output_file, 'w', encoding='UTF-8') as f:
    for item in org_data:
        f.write(item['chatgpt_zeroshot'].replace('\n', '\\n').replace('\r', '\\r') + '\n')

Processing documents: 100%|██████████| 1831/1831 [21:16<00:00,  1.43it/s]
